<a href="https://colab.research.google.com/github/ankitshripalsingh/upGrad/blob/OR/Airlines_Optimisation_stub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Airlines price optimisation problem

## Problem Statement

FlyIndia is a fictional airlines company. FlyIndia would have two types of customer segments to cater to the 'early birds', i.e. the customers who would be buying tickets much in advance and, hence, would be eligible for a discount, and the 'late buyers' or the ones who would be buying the tickets at the regular prices. The airline company wants to maximise its revenue for a particular Delhi to Bangalore flight by allocating the regular and discounted seats judiciously. There are 166 seats available in the air craft. Based on the past observations, regular demand and the discounted demand do not exceed 100 and 150 tickets respectively.

The prices of the tickets are as follows:

`Discounted:` ₹1190 \
`Regular (Non-Discounted):` ₹3085




![Airlines_image.png](https://github.com/ankitshripalsingh/upGrad/blob/OR/Airlines_image.png?raw=1)

### Step1:
<b> Import Pyomo Environment </b>

In [1]:
# Ask Python to load the Pyomo modelling environment
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
!apt-get install -y -qq glpk-utils
from pyomo.environ import *

     |████████████████████████████████| 9.6 MB 35.0 MB/s 
     |████████████████████████████████| 49 kB 8.2 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to un

### Step2:
<b>Specify / import data</b>

**Sets:**

Sets are used to define the indexes for an optimisation problem. There are two ways to define the indexes.

**General syntax:**

- Python lists/sets containing the indexes 

- `Set()` class of Pyomo

    > model.id = `Set`(initialize = [1,2,3]) <br>
    >  model.id = `Set`( initialize=range(5) ) <br>
    > model.id = `RangeSet`(5) <br>


<b>Note:</b>
- Python range is 0-based and gives [ 0, 1, 2, 3, 4 ]
- RangeSet is 1-based and gives [ 1, 2, 3, 4, 5 ]



In [2]:
# Indexes - Defining sets/lists containing the indexes for this problem. Type of ticket becomes the index here. 
# Early bird- 'eb', Regular - 'reg'
tkt_type = ['eb', 'reg']

There are two ways to define the parameters for a given optimisation problem.

<b>General syntax:</b> <br>

- Python dictionary with keys as indexes and values as corresponding parameter values <br>
    >data_dict = {'index_1' : 7, 'index_2' : 5}
    

-  `Param()` class <br>
    > model.x_param = `Param`(index_values, initialize = data_dict)<br>
    > model.x_param = `Param`(index_values, initialize = data_dict, default=0, mutable=True)<br>

<b>Note:</b> <br>
- index_values is a Python set/list or a Pyomo set containing the indexes for the parameters/decision variables.
- data_dict is a Python dictionary with keys as indexes and values as corresponding parameter values.
- Other arguments to `Param()` class are `default` and `mutable`.
- Providing `'default'` allows the data initialised to only  specify the “unusual” values.
- `'mutable=True'` indicates that the parameter can be changed during run-time.

    


In [10]:
# parameters - price of each ticket type is defined as a Python dictionary (method 1 explained above)
tkt_price = {'eb': 1190, 'reg': 3085}
tkt_demand = {'eb': 150, 'reg': 100}

### Step3:
<b> Create Model Object</b> <br>

- Pyomo allows you to build two types of model – `concrete` / `abstract` <br>
    - `Concrete` model is used when all the data required for buiding a model are available before-hand <br>
    - `Abstract` model is used when some of the data are fed during the runtime



In [11]:
# Creating an instance of a Concrete model since we have all the required data before hand
model = ConcreteModel()

### Step4: 

<b>Define Decision Variable</b>

<b>General syntax:</b> <br>
- model.a_variable = `Var`(within = NonNegativeReals)
- model.a_variable = `Var`(domain = NonNegativeReals)
- model.a_variable = `Var`(bounds= (0,60))
- model.a_variable = `Var`(initialize = 10, bounds= (0,60))

<b>Note:</b> <br>
- 'within' is an optional argument. The keyword 'domain' is an alias for 'within'.
- Other possible domains are PositiveIntegers, Reals, Binary etc.


In [12]:
#variables - decision variable X that has the index as tkt_types
model.X = Var(tkt_type, within = PositiveIntegers)

### Step5:
<b>Define Objective</b>

<b>General syntax:</b> <br>
- model.obj = `Objective`(expr = 3* model.x +4*model.y,  sense = minimize) <br>

- model.obj = `Objective`(rule = obj_rule,  sense = minimize )

    def obj_rule(m):
        return(3* model.x +4*model.y)
 
- model.value = `Objective`(expr = sum (a[i]*model.x[i] for i in id), sense=maximize)

<b>Note:</b> <br>
- The default sense is `'minimum'`
- The keyword argument `'expr'` can either be assigned an expression or any function-like object that returns an expression 






In [13]:
#Objective - to maximise the total revenue
model.value = Objective(expr = sum(tkt_price[i]*model.X[i] for i in tkt_type), sense = maximize)

### Step6:
<b>Define Constraints</b>

<b>General syntax:</b> <br>
- model.constraint_1 = `Constraint`(expr = model.x + 5 * model.y >= 5 ) <br>

- model.constraint_1 = `Constraint`(rule = constraint_rule)

    def constraint_rule(m):
        return(model.x + 5 * model.y >= 5)

<b>Note:</b> <br>

- The keyword argument `'expr'` can either be assigned an expression or any function-like object that returns an expression.
- `'expr'` can also be a tuple as shown below:
    - model.obj = `Constraint`(expr = (None, model.x + model.y, 1))
    - 3-tuple specifies ( lower_bound, expr, upper_bound )
    - 2-tuple specifies an equality constraint.
   




In [14]:
#constraints

#expr method

#Regular demand doesn't exceed 100
model.reg_demand = Constraint(expr = model.X['reg'] <= tkt_demand['reg'])

#Discount demand doesn't exceed 150
model.eb_demand = Constraint(expr = model.X['eb'] <= tkt_demand['eb'])

#rule method
def supply_constraint(model):
  return(sum(model.X[i] for i in tkt_type) == 166)

#Maximum seats available is 166
model.supply = Constraint(rule = supply_constraint)

### Step7:
<b>Create solver & solve model </b><br>

It is important to know if you have created a linear(LP), integer(IP), mixed integer(MIP), non-linear (NLP), or mixed integer non-linear (MINLP) model and choose the most suitable solver accordingly. We use glpk for LP, IP and MIP type problems and ipopt for NLP type problems.

In [15]:
result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 387040.0
  Upper bound: 387040.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.012943029403686523
# ----------------------------------------------------------
#   Solution Information
# ------------------------------

### Step8:
<b>Display Results </b>

In [16]:
model.pprint()

1 Set Declarations
    X_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'eb', 'reg'}

1 Var Declarations
    X : Size=2, Index=X_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         eb :     1 :  66.0 :  None : False : False : PositiveIntegers
        reg :     1 : 100.0 :  None : False : False : PositiveIntegers

1 Objective Declarations
    value : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 1190*X[eb] + 3085*X[reg]

3 Constraint Declarations
    eb_demand : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : X[eb] : 150.0 :   True
    reg_demand : Size=1, Index=None, Active=True
        Key  : Lower : Body   : Upper : Active
        None :  -Inf : X[reg] : 100.0 :   True
    supply : Size=1, Index=None, Active=True
        Key  : Lower : Body           : Upper

In [17]:
#Print the value of the objective function

model.value()

387040.0